In [1]:
import os
import numpy as np
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4041 --packages org.apache.kafka:kafka_2.11:0.9.0.0,org.apache.kafka:kafka-clients:0.9.0.0  pyspark-shell'

In [2]:
from pyspark import SparkContext
sc = SparkContext("local[1]", "KafkaSendStream") 
from kafka import KafkaProducer
import time

In [24]:
import json
from satori.rtm.client import make_client, SubscriptionMode
from math import log

# Make a proxy class

endpoint = "wss://open-data.api.satori.com"
appkey = "dC6c33Fbb5ECdAC1Ef2aB77dcBfBB0B0"
channel = "cryptocurrency-market-data"

with make_client(endpoint=endpoint, appkey=appkey) as client:
    print('Connected to Satori RTM!')
    messagebox=[]

    class SubscriptionObserver(object):
        def on_subscription_data(self, data):
            for message in data['messages']:
                messagebox.append(message)

    subscription_observer = SubscriptionObserver()
    client.subscribe(
        channel,
        SubscriptionMode.SIMPLE,
        subscription_observer,
        {'filter':'select * from `cryptocurrency-market-data` where exchange = "Bitstamp" and basecurrency = "USD" and cryptocurrency="BTC"'})
        #  exchange != "ShapheShift" OU type != "ratequote" si on laisse ShapeShift

    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    i=0
    n=4   # number of inputs
    numOfRendements=2
    while not messagebox: # waiting for a first price to arrive
        time.sleep(1)
    lastPrice=float(json.loads(json.dumps(messagebox.pop(), ensure_ascii=False))['price']) # get that first price
    proxies=np.empty(0) # x of the RLS algorithm
    while proxies.size<n+1:
        rendements=np.empty(0) 
        while rendements.size<numOfRendements: # Choix du nombre de rendements pour le calcul du proxy            
            if messagebox:
                message=messagebox.pop()
                message=json.dumps(message, ensure_ascii=False)
                message=json.loads(message)
                rendement=np.log(float(message['price'])/lastPrice) # r = np.log(float(message['price'])/lastPrice)
                print(float(message['price']), lastPrice, rendement)
                lastPrice=float(message['price'])
                rendements=np.append(rendements, rendement)
                #print(float(message['price']), lastPrice, log(float(message['price'])/lastPrice))print(rendements, rendements.size)
            time.sleep(1)
        proxies=np.append(proxies, np.sum(np.square(rendements))/numOfRendements)
    
    y=proxies[-1]
    proxies=np.delete(proxies, proxies.size-1)
    message=np.array2string(np.append([i,y], proxies),separator=",") 
    producer.send('test2', message)
    i=i+1
    print(message)
    """
        while True:
            proxies=np.roll(proxies,-1)
            proxies[0]=1
            while not messagebox:
                time.sleep(1)
            message=messagebox.pop()
            message=json.dumps(message, ensure_ascii=False)
            message=json.loads(message)
            proxy+=(np.log(float(message['price'])/lastPrice)**2)/j # r = np.log(float(message['price'])/lastPrice)
            lastPrice=float(message['price'])
            proxies[n-1]=proxy
            j+=1
            y=np.sum(proxies)+0.1*np.random.rand(1)[0]
            message=np.array2string(np.append([i,y], proxies),separator=",") 
            producer.send('test2', message)
            i=i+1
            print(message)
    """

Connected to Satori RTM!
(7775.0, 7779.91, -0.00063131193469776153)
(7775.0, 7775.0, 0.0)
(7775.0, 7775.0, 0.0)
(7770.0, 7775.0, -0.00064329368574170764)
(7756.43, 7770.0, -0.0017479875870004861)
(7746.66, 7756.43, -0.0012603940625230529)
(7749.99, 7746.66, 0.00042977031180991838)
(7749.99, 7749.99, 0.0)
(7760.0, 7749.99, 0.0012907811532847995)
(7750.0, 7760.0, -0.0012894908298717507)
[  0.00000000e+00,  1.66445129e-06,  1.99277379e-07,  2.06913383e-07,
   2.32202690e-06,  9.23512605e-08]
